In [ ]:
import numpy as np
from sklearn.datasets import load_diabetes

X_diab, y_diab = load_diabetes(return_X_y=True) # returns diabetes data shapes: (442, 10) and (442,)

In [ ]:
from sklearn.datasets import make_regression

X_reg, y_reg = make_regression(n_samples=60, n_features=10, noise=0.5, random_state=42)

In [52]:
class Layer:
    def __init__(
            self, 
            units, 
            *, 
            input_layer: bool = False,
            activation: str = "linear",
            use_bias: bool = True,
            # optimizer: str = "gd" # Testing
            ):
        """
        Initialize a neural network layer.

        Args:
            units (int): Count of neurons in the layer.
            input_layer (bool, optional): Whether the layer is an input layer. Defaults to False.
            activation (str, optional): Activation function for the layer. Can be "linear", "relu", or "sigmoid". Defaults to "linear".
            use_bias (bool, optional): Whether to use bias in the layer. Defaults to True.
        """
            
        
        self.units = units
        self.input_layer = input_layer
        self.activation = activation
        self.use_bias = use_bias
        self.optimizer = None # optimizer # Different optimizers for layers

        self._input = None
        self._output = None

        self.w = None # Weights matrix
        self._weight_gradient = None # Weight derivative matrix
        self._bias_gradient = None # Bias derivative vector

        # self._adagrad_weight = None # AdaGrad optimizer's r for weight
        # self._adagrad_bias = None # AdaGrad optimizer's r for bias

    def activationFunction(self, z):
        """
        Apply the activation function to the given input.

        Args:
            z (numpy.ndarray): Input to the activation function.

        Returns:
            numpy.ndarray: Output after applying the activation function.
        """

        if self.activation == "linear":
            return z

        if self.activation == "relu":
            return np.maximum(z, np.zeros(z.shape))

        if self.activation == "sigmoid":
            return 1 / (1 + np.exp(-z))

    def _weightInit(self, input_size):
        """
        Initialize the weights matrix based on the input size.

        Args:
            input_size (int): Size of the input.

        Notes:
            Only executed for layers other than the input layer.
        """

        if self.input_layer:
            return # input_layer doesn't need weights

        self.w = np.random.normal(loc = 0, scale = 1 / input_size, size=(input_size, self.units)) # loc -> mean, scale -> variance
        self.bias = np.zeros((1, self.units))

    def _setOptimizer(self, optimizer):

        # TODO:
        # check valid
        #####

        if self.input_layer:
            return

        self.optimizer = optimizer

        if self.optimizer == "sgd":
            self.batch_size = 1 # SGD is the same as mini-batch gradient descent when batch-size = 1

        if self.optimizer == "adagrad":
            self._adagrad_weight = np.zeros(self.w.shape)

            if self.use_bias:
                self._adagrad_bias = np.zeros(self.bias.shape)

    def _activationDerivative(self):
        """
        Compute the derivative of the activation function.

        Returns:
            numpy.ndarray: Derivative of the activation function.

        Notes:
            Only supports the "linear", "relu", and "sigmoid" activation functions.
        """

        if self.activation == "linear":
            return 1

        if self.activation == "relu":
            return (self._output > 0) * 1

        if self.activation == "sigmoid":
            return self._output * (1 - self._output)

    def _setGrad(self, grad):
        """
        Calculate the gradients of weights and bias for backpropagation.

        Args:
            grad (numpy.ndarray): Gradient from the previous layer.

        Returns:
            numpy.ndarray: Gradient to be passed to the previous layer.

        Notes:
            Only executed for layers other than the input layer.
        """

        if self.input_layer:
            return
        
        grad = grad * self._activationDerivative()
        self._weight_gradient = self._input.T @ grad

        if self.use_bias:
            self._bias_gradient = grad.sum(axis=0, keepdims=True)

        return grad @ self.w.T
    
    def _updateGrad(self, learning_rate):
        """
        Update the weights and bias based on the computed gradients.

        Args:
            learning_rate (float): Learning rate for gradient descent.

        Notes:
            Only executed for layers other than the input layer.
        """

        if self.input_layer:
            return

        eps = 10e-8

        if self.optimizer == "gd":
            self.w -= learning_rate * self._weight_gradient
            if self.use_bias:
                self.bias -= learning_rate * self._bias_gradient

        if self.optimizer == "adagrad":
            self._adagrad_weight += self._weight_gradient ** 2
            learning_rate_weight = learning_rate / ( np.sqrt(self._adagrad_weight) + eps)
            self.w -= learning_rate * self._weight_gradient

            if self.use_bias:
                self._adagrad_bias += self._bias_gradient ** 2
                learning_rate_bias = learning_rate / ( np.sqrt(self._adagrad_bias) + eps)

                self.bias -= learning_rate * self._bias_gradient


    def call(self, X):
        """
        Perform a forward pass through the layer.

        Args:
            X (numpy.ndarray): Input to the layer.

        Returns:
            numpy.ndarray: Output of the layer after applying the activation function.
        """
        if self.input_layer:
            return X
        
        self._input = X
        self._output = self.activationFunction(X @ self.w + self.bias)

        return self._output

In [58]:
class NeauralNetwork:
    def __init__(
            self, 
            layers: list, 
            loss_function: str = "mse", 
            learning_rate = 0.01,
            verbose: bool = False,
            optimizer: str = "gd",
            epochs: int = 1, 
            batch_size: int = 32):
        """
        Initialize a neural network.

        Args:
            layers (list): List of Layer objects defining the network architecture. 
            loss_function (str, optional): Loss function to use. Defaults to "mse".
            optimizer (str, optional): Optimization algorithm to use for updating weights during training.
                Options include:
                - "gd" (Gradient Descent): Standard gradient descent.
                // - "sgd" (Stochastic Gradient Descent): Update weights using a single sample at a time.
                // - "adam" (Adam): Adaptive Moment Estimation algorithm.
                Defaults to "gd".

            learning_rate (float, optional): Learning rate for gradient descent. Defaults to 0.01.
            max_iter (int, optional): Maximum number of iterations for training. Defaults to 1000.
            verbose (bool, optional): Whether to display training progress. Defaults to False.

            // TODO: Documentation
            epochs:
            batch_size:
        """

        self.layers = layers
        self.loss_function = loss_function
        self.learning_rate = learning_rate
        self.verbose = verbose
        self.optimizer = optimizer # Optimizer for all layers
        self.epochs = epochs
        self.batch_size = batch_size

        # Weights initializing:
        for i in range(len(self.layers)):
            self.layers[i]._weightInit(self.layers[i - 1].units)
            self.layers[i]._setOptimizer(self.optimizer)



    def lossFunction(self, y_true, y_pred):
        """
        Compute the loss between the true values and predicted values.

        Args:
            y_true (numpy.ndarray): True values.
            y_pred (numpy.ndarray): Predicted values.

        Returns:
            float: Loss value.
        """

        if self.loss_function == "mse":
            return 0.5 * np.mean(np.linalg.norm(y_pred - y_true, axis=1)**2)

        # Can be add

    def _lossFunctionDerivative(self, y_pred, y_true):
        """
        Compute the derivative of the loss function.

        Args:
            y_pred (numpy.ndarray): Predicted values.
            y_true (numpy.ndarray): True values.

        Returns:
            numpy.ndarray: Derivative of the loss function.
        """

        if self.loss_function == "mse":
            return 1 / len(y_pred) * (y_pred - y_true)

        # Can be add

    def fit(self, X, y):
        """
        Train the neural network on the given input-output pairs.

        Args:
            X (numpy.ndarray): Input data.
            y (numpy.ndarray): Output data.

        Notes:
            Reshapes X and y to match the expected input shapes of the network.
        """

        batch_separation = [(X[i:i + self.batch_size], y[i:i + self.batch_size]) for i in range(0, 442, self.batch_size)]
        epoch_len = len(batch_separation)

        for _ in range(self.epochs):       

            for iter, (X_, y_) in enumerate(batch_separation):

                pred = self.forward(X_)

                if self.verbose:
                    process_percent = int(iter / epoch_len * 10)
                    print(f"\r Epoch {_ + 1}/{self.epochs}; Batch {iter}/{epoch_len}: [{process_percent * '=' + '>' + (10 - process_percent) * '-'}] - loss: {self.lossFunction(y_, pred)}",end='')
                
                self.backward(pred, y_)
            
            if self.verbose:
                print(f"\r Epoch {_ + 1}/{self.epochs}; Batch {iter + 1}/{epoch_len}: [{11 * '='}] - loss: {self.lossFunction(y_, pred)}")

    def predict(self, X):
        """
        Perform predictions using the trained neural network.

        Args:
            X (numpy.ndarray): Input data.

        Returns:
            numpy.ndarray: Predicted output data.
        """

        return self.forward(X)
        
    def forward(self, X):
        """
        Perform a forward pass through the network.

        Args:
            X (numpy.ndarray): Input data.

        Returns:
            numpy.ndarray
        """

        X_ = np.copy(X)
        
        for layer in self.layers:
            X_ = layer.call(X_)
        return X_

    def backward(self, y_pred, y_true):
        """
        Perform backpropagation to update the weights of the network.

        Args:
            y_pred (numpy.ndarray): Predicted values.
            y_true (numpy.ndarray): True values.
        """
        
        gradient = self._lossFunctionDerivative(y_pred, y_true)

        for layer in reversed(self.layers):
            gradient = layer._setGrad(gradient)
            layer._updateGrad(self.learning_rate)

In [59]:
nn = NeauralNetwork(layers=[
        Layer(units=10, input_layer=True),
        # Layer(units=40, activation="sigmoid"),
        Layer(units=40, activation="relu"),
        Layer(units=40, activation="relu"),
        Layer(units=1),
    ],
    loss_function = "mse",
    learning_rate=0.001, 
    verbose=True,
    optimizer="adagrad",
    batch_size = 32,
    epochs=1000
)
y_diab = y_diab.reshape(-1, 1)
nn.fit(X_diab, y_diab)

 Epoch 1/1000; Batch 14/14: [===========] - loss: 10481.479467102554
 Epoch 2/1000; Batch 14/14: [===========] - loss: 9012.0816387569567
 Epoch 3/1000; Batch 14/14: [===========] - loss: 5358.0644728852344
 Epoch 4/1000; Batch 14/14: [===========] - loss: 5100.1589742471954
 Epoch 5/1000; Batch 14/14: [===========] - loss: 4673.3088579786547
 Epoch 6/1000; Batch 14/14: [===========] - loss: 4055.1930844295853
 Epoch 7/1000; Batch 14/14: [===========] - loss: 3164.0825884961264
 Epoch 8/1000; Batch 14/14: [===========] - loss: 2346.0512125749497
 Epoch 9/1000; Batch 14/14: [===========] - loss: 1779.6442340043047
 Epoch 10/1000; Batch 14/14: [===========] - loss: 1399.7322654347777
 Epoch 11/1000; Batch 14/14: [===========] - loss: 5118.1629465739817
 Epoch 12/1000; Batch 14/14: [===========] - loss: 4149.0101599395123
 Epoch 13/1000; Batch 14/14: [===========] - loss: 3578.0501467242514
 Epoch 14/1000; Batch 14/14: [===========] - loss: 3008.4908585052933
 Epoch 15/1000; Batch 14/14: 

In [54]:
nn.predict(X_diab)

array([[-3.00863455e-04],
       [-1.28876178e-04],
       [-3.37749660e-04],
       [-7.73479155e-05],
       [ 8.18006031e-06],
       [-2.12885040e-04],
       [-4.03425618e-04],
       [-9.79001148e-05],
       [-2.37367204e-04],
       [-3.99828573e-05],
       [-3.15673745e-04],
       [-3.61422914e-04],
       [-1.52242264e-05],
       [-1.42808485e-04],
       [-3.35033253e-05],
       [-1.78867959e-04],
       [ 6.54289830e-05],
       [-9.84896127e-05],
       [-1.04739105e-04],
       [-2.75358676e-05],
       [-1.70954311e-04],
       [-3.92214966e-04],
       [-2.10782776e-04],
       [-3.55347262e-04],
       [-1.34492732e-04],
       [-3.12458788e-04],
       [-3.13882001e-04],
       [-1.86040192e-04],
       [-1.44840732e-04],
       [-2.11750346e-04],
       [-1.22677384e-04],
       [-1.33908463e-04],
       [-3.44780867e-04],
       [-1.89532739e-04],
       [-1.39426663e-04],
       [-3.11759788e-04],
       [ 7.68526227e-05],
       [-4.86714102e-05],
       [-1.2

In [ ]:
import tensorflow as tf

alg = tf.keras.Sequential([
    tf.keras.layers.Input(10),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(1, activation='relu'),
])

alg.compile(optimizer='adam', loss='mse')

alg.fit(X_diab, y_diab, epochs=1000, batch_size=64)